In [5]:
import pandas as pd
import os
import numpy as np
import scipy as sp
import sklearn.ensemble as ske
from sklearn.cross_validation import cross_val_score
import tensorflow as tf
from __future__ import print_function
import inspect

D:\Anaconda\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Prelim Analysis

In [6]:
os.chdir('./Kaggle_Digit_Recognizer/')

In [7]:
train = pd.read_csv('train.csv')

In [8]:
test = pd.read_csv('test.csv')

In [9]:
Results=pd.DataFrame(data={'ImageId':test.index+1})

In [10]:
train['label'] = train['label'].astype('category')

In [11]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Simplify By changing images from 0's and 255's to 0's and 1's

In [12]:
train.ix[:,1:]=train.ix[:,1:]/255.

D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [13]:
max(train.ix[1,:].as_matrix())

1.0

In [14]:
train.shape

(42000, 785)

In [15]:
test.ix[:,:]=test.ix[:,:]/255.

D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [16]:
max(test.ix[1,:].as_matrix())

1.0

In [17]:
test.shape

(28000, 784)

# Random Forest Classifier

In [18]:
clf=ske.RandomForestClassifier(n_estimators=100)

In [19]:
scores = cross_val_score(clf, train.drop('label',axis=1), train['label'])

In [20]:
scores.mean()

0.96211925692734734

In [21]:
clf = clf.fit(train.drop('label',axis=1), train['label'])

In [22]:
Results['Label']=clf.predict(test)

In [23]:
Results.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [24]:
Results[['ImageId','Label']].to_csv('RFpred.csv',index=False)

In [25]:
Results.rename(columns={'Label': 'RFpred'}, inplace=True)

Test Predictions were 96.457% Correct

# TensorFlow MLP

In [26]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [27]:
np.shape(mnist.train.labels)

(55000, 10)

In [28]:
mnist.train.labels

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

First I want to make one_hot vectors

In [29]:
labels = train.label.as_matrix()

In [30]:
len(np.unique(labels))

10

In [31]:
np.shape(labels)

(42000,)

In [32]:
ohlabels = np.zeros((42000,10))
ohlabels[np.arange(42000),labels]=1

In [33]:
ohlabels

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [34]:
images = train.drop('label',axis=1).as_matrix()

In [48]:
learning_rate = .001
training_epochs = 10 #?
batch_size = 100
display_step = 20

n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10

In [36]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [37]:
trainlist=train.as_matrix()

In [38]:
#Create simple MLP
def mlp(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [39]:
weights = {
    'h1': tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2,n_classes]))
}

In [40]:
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [41]:
pred = mlp(x, weights, biases)

In [45]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=pred))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [46]:
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
#Modify below code to work with my dataset
#Get it to predict

In [49]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost=0.
        total_batch = int(np.shape(images)[0]/batch_size)
        np.random.shuffle(trainlist)
        for i in range(total_batch):
            batch_x = trainlist[:,1:][i*batch_size:(i+1)*batch_size]
            lbl_y = trainlist[:,0][i*batch_size:(i+1)*batch_size]
            batch_y = np.zeros((batch_size,10))
            batch_y[np.arange(batch_size),lbl_y.astype('int')]=1
            _, c = sess.run([optimizer,cost],feed_dict={x:batch_x, y:batch_y})
            avg_cost += c / total_batch
        if epoch % display_step ==0:
            print("Epoch:", '%04d' % (epoch+1),"cost=", "{:.9f}".format(avg_cost))
    print("Optimization finished")
    correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    predict=tf.argmax(pred,1)
    pred_labels=predict.eval(feed_dict={x: test.as_matrix()})
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 195.693373934
Optimization finished
Accuracy: 0.9595


In [50]:
Results['Label']=pred_labels

In [51]:
Results[['ImageId','Label']].to_csv('NNpred.csv',index=False)

In [52]:
Results.rename(columns={'Label': 'NNpred'}, inplace=True)

Test Predictions were 96.9% correct

# TensorFlow CNN

fairly brief introduction @ http://www.kdnuggets.com/2015/04/inside-deep-learning-computer-vision-convolutional-neural-networks.html

Greater understanding of the convolutional layer in TensorFlow can be found @ http://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow



In [53]:
learning_rate = .0001
training_epochs=30
training_iters = 2500 #?
batch_size = 50
display_step = 1


n_input = 784
n_classes = 10
dropout = .5

In [54]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
#keep_prob placeholder needed b/c we're doing dropout
keep_prob = tf.placeholder("float")

In [55]:
# define convolutional layer
# The changeable values strides correspond to the the stride taken by the filter window each move
# Thus default is the filter only takes a stride of 1 each time
# This function:
    # Runs filter W over x(an image) based on "strides" parameters to produce new image
    # The new image is created by the dot product between a filter and the collection of pixels the filter currently "covers"
    # adds bias to the image
    # Gives new image to a rectified linear unit(which is an activation function of type f(x) = max(0,x)) 
    # and returns the output
def conv2d(x,W,b,strides=1):
    x = tf.nn.conv2d(x,W, strides=[1,strides,strides,1],padding="SAME")
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [56]:
# pooling reduces dimensionality
# ksize represents size of pooling window and strides represents how much it moves every time
# For each stop the pooling window chooses only the max pixel from what it covers, then moves on
# Thus this pooling will find the max pixel of the 4(2x2) pixels it looks at each time
def maxpool2d(x,k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1],strides = [1,k,k,1], padding='SAME')

In [57]:
# Creating Convolutional neural net with 2 Conv. and pooling layers, and A Fully connection layer w/ dropout
def conv_net(x, weights, biases, dropout):
    #x inputed as a matrix of 1x784 pixels, want to change it to 28x28 pixels
    x=tf.reshape(x, shape=[-1,28,28,1]) # Why is this -1?
    # First Conv. layer with pooling
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1,k=2)
    # Second Conv. layer with pooling
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2,k=2)
    #Fully-Connected Layer
    fc1 = tf.reshape(conv2, [-1,weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


In [58]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

In [59]:
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [60]:
pred = conv_net(x,weights,biases,keep_prob)

In [62]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
#cost = -tf.reduce_sum(y*tf.log(pred))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [63]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [64]:
# Initializing the variables
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


running_index=0
trainlist=train.as_matrix()
img_vals=trainlist[:,0]
training_imgs = np.zeros((train.shape[0],n_classes))
training_imgs[np.arange(train.shape[0]),img_vals.astype('int')]=1
training_lbls=trainlist[:,1:]
def next_batch(batch_size):
    global running_index
    global training_imgs
    global training_lbls
    start=running_index
    running_index+=batch_size
    if running_index > train.shape[0]:
        perm = np.arange(train.shape[0])
        np.random.shuffle(perm)
        training_imgs=training_imgs[perm]
        training_lbls=training_lbls[perm]
        start=0
        running_index=batch_size
    end = running_index
    return training_lbls[start:end],training_imgs[start:end]
        

In [65]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost=0
        avg_acc=0
        # Keep training until reach max iterations
        total_batch = int(np.shape(images)[0]/batch_size)
        np.random.shuffle(trainlist)
        for i in range(total_batch):
            batch_x = trainlist[:,1:][i*batch_size:(i+1)*batch_size]
            lbl_y = trainlist[:,0][i*batch_size:(i+1)*batch_size]
            batch_y = np.zeros((batch_size,10))
            batch_y[np.arange(batch_size),lbl_y.astype('int')]=1
            # Run optimization op (backprop)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                           keep_prob: dropout})
            if epoch % display_step == 0:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                                  y: batch_y,
                                                                  keep_prob: 1.})
                avg_cost += loss / total_batch
                avg_acc += acc / total_batch
        if epoch % display_step == 0:
            print("Iter " + str(epoch) + ", Minibatch Loss= " + \
                "{:.6f}".format(avg_cost) + ", Training Accuracy= " + \
                "{:.5f}".format(avg_acc))
    print("Optimization Finished!")
    print("Predicting Testing Set Classes...")
    predict= tf.argmax(pred,1)
    pred_labels = np.zeros(test.shape[0])
    testmatrix=test.as_matrix()
    for j in range(int(test.shape[0]/batch_size)):
        if j % 56 == 0:
            print(str((j/(test.shape[0]/batch_size))*100) + "% Done")
        pred_labels[j*batch_size:(j+1)*batch_size] = predict.eval(feed_dict = {x: testmatrix[j*batch_size:(j+1)*batch_size],keep_prob:1.0})

Iter 0, Minibatch Loss= 11780.484097, Training Accuracy= 0.61388
Iter 1, Minibatch Loss= 2282.959498, Training Accuracy= 0.87433
Iter 2, Minibatch Loss= 1456.050982, Training Accuracy= 0.91202
Iter 3, Minibatch Loss= 1065.244949, Training Accuracy= 0.92929
Iter 4, Minibatch Loss= 813.728931, Training Accuracy= 0.93938
Iter 5, Minibatch Loss= 636.396363, Training Accuracy= 0.94674
Iter 6, Minibatch Loss= 502.320066, Training Accuracy= 0.95269
Iter 7, Minibatch Loss= 409.251078, Training Accuracy= 0.95681
Iter 8, Minibatch Loss= 334.412384, Training Accuracy= 0.96060
Iter 9, Minibatch Loss= 279.558497, Training Accuracy= 0.96326
Iter 10, Minibatch Loss= 237.491233, Training Accuracy= 0.96557
Iter 11, Minibatch Loss= 205.497062, Training Accuracy= 0.96669
Iter 12, Minibatch Loss= 175.805714, Training Accuracy= 0.96910
Iter 13, Minibatch Loss= 151.423010, Training Accuracy= 0.97012
Iter 14, Minibatch Loss= 131.164580, Training Accuracy= 0.97219
Iter 15, Minibatch Loss= 113.734446, Training

In [66]:
Results['Label']=pred_labels.astype('int')
Results[['ImageId','Label']].to_csv('CNpred.csv',index=False)
Results.rename(columns={'Label': 'CNNpred'}, inplace=True)

20 epochs; batch size of 50  = 96.586% Correct

In [116]:
train.shape

(42000, 785)